In [20]:
import numpy as np
import scipy.stats as stats

In [21]:
import lightgbm as lgbm

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [23]:
from sklearn.model_selection import RandomizedSearchCV

In [24]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [25]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [26]:
processed = pd.read_csv('features_13.csv')

In [32]:
cfeats = []
for i,col in enumerate(processed.columns):
    if processed[col].dtype.name=='object':
        processed[col] = processed[col].astype('category')

In [33]:
processed = processed.set_index(processed.columns[0])
processed.head()

,mayor_evento,dispos,primer_visita,ultima_visita,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,region,...,Computer_bool,Tablet_bool,Paid_bool,Organic_bool,Direct_bool,Referral_bool,Social_bool,Email_bool,primer_visita_seconds,ultima_visita_seconds
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Computer,05-17,05-17,6,1,14,6.0,Referral,Unknown,...,True,False,False,False,False,True,False,False,44867,59317
00091926,viewed product,Computer,05-03,05-31,448,28,0,16.0,Direct,Rio Grande do Sul,...,True,False,True,False,True,True,False,False,79709,71523
00091a7a,brand listing,Smartphone,03-26,03-26,10,1,66,10.0,Paid,Minas Gerais,...,False,False,True,False,False,False,False,False,53471,53818
000ba417,viewed product,Computer,05-17,05-26,206,10,5,20.6,Direct,Sao Paulo,...,True,False,False,False,True,False,False,False,40305,47362
000c79fe,searched products,Smartphone,05-29,05-29,17,1,2,17.0,Paid,Minas Gerais,...,False,False,True,False,False,False,False,False,1667,2287


In [34]:
predict_features = processed.loc[processed.index.isin(predict['person'])].drop(['primer_visita','ultima_visita'],axis=1)

In [35]:
predict_features.head()

,mayor_evento,dispos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,region,personas_mismo_prod_visto,opsis,...,Computer_bool,Tablet_bool,Paid_bool,Organic_bool,Direct_bool,Referral_bool,Social_bool,Email_bool,primer_visita_seconds,ultima_visita_seconds
person,,,,,,,,,,,,,,,,,,,,,
00091926,viewed product,Computer,448,28,0,16.000000,Direct,Rio Grande do Sul,2889,Windows 7,...,True,False,True,False,True,True,False,False,79709,71523
00091a7a,brand listing,Smartphone,10,1,66,10.000000,Paid,Minas Gerais,2889,Android 5.1.1,...,False,False,True,False,False,False,False,False,53471,53818
000ba417,viewed product,Computer,206,10,5,20.600000,Direct,Sao Paulo,74,Windows 10,...,True,False,False,False,True,False,False,False,40305,47362
000e4d9e,viewed product,Computer,411,15,1,27.400000,Referral,Minas Gerais,901,Windows 10,...,True,False,True,True,True,True,False,False,42895,63922
000e619d,viewed product,Computer,68,3,13,22.666667,Organic,Sao Paulo,785,Windows 7,...,True,False,True,True,True,False,False,False,66008,59138


In [36]:
training_features = processed.loc[processed.index.isin(a_predecir['person'])]
training_features['label'] = a_predecir.set_index('person')['label']


training_labels = training_features['label']
training_features = training_features.drop(['primer_visita','ultima_visita','label'],axis=1)

C:\Users\Usuario\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [37]:
training_features.head()

,mayor_evento,dispos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,region,personas_mismo_prod_visto,opsis,...,Computer_bool,Tablet_bool,Paid_bool,Organic_bool,Direct_bool,Referral_bool,Social_bool,Email_bool,primer_visita_seconds,ultima_visita_seconds
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Computer,6,1,14,6.000000,Referral,Unknown,1699,Windows 10,...,True,False,False,False,False,True,False,False,44867,59317
000c79fe,searched products,Smartphone,17,1,2,17.000000,Paid,Minas Gerais,1404,Android 7,...,False,False,True,False,False,False,False,False,1667,2287
001802e4,ad campaign hit,Smartphone,19,1,0,19.000000,Paid,Rio de Janeiro,2889,Android 7.1.1,...,False,False,True,False,False,False,False,False,63263,63909
0019e639,viewed product,Computer,471,115,9,4.095652,Paid,Sao Paulo,785,Windows 7,...,True,False,True,True,False,True,False,False,51960,80157
001b0bf9,viewed product,Ninguno,7,1,9,7.000000,Ninguno,Unknown,553,Ninguno,...,False,False,False,False,False,False,False,False,51332,52250


In [38]:
params = {
    'num_leaves':stats.randint(90,5000),
    'min_data_in_leaf':stats.randint(100,1000),
    'max_bin':stats.randint(70,300),
    'learning_rate':stats.uniform(0.01,1-0.01),
    'n_estimators':stats.randint(150,500)
}
grid = RandomizedSearchCV(lgbm.LGBMClassifier(boosting_type='dart'),
                          param_distributions=params,
                          scoring='roc_auc',
                          cv=2,
                          verbose=1,
                          n_iter=150)

In [39]:
xtr,xte,ytr,yte=train_test_split(training_features,training_labels)

In [40]:
grid.fit(xtr,ytr)

Fitting 2 folds for each of 150 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  8.8min finished


RandomizedSearchCV(cv=2, error_score='raise-deprecating',
          estimator=LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
          fit_params=None, iid='warn', n_iter=150, n_jobs=None,
          param_distributions={'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E13977AD30>, 'min_data_in_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E1336C3A90>, 'max_bin': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E1332771D0>, 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001E139B535C0>, 'n_estimators': <scipy.stats._distn_

In [41]:
grid.score(X=xte,y=yte)

0.8613224388904147

In [42]:
grid.best_params_

{'learning_rate': 0.037644697126700453,
 'max_bin': 249,
 'min_data_in_leaf': 122,
 'n_estimators': 359,
 'num_leaves': 1447}

## Prediccion

In [43]:
predictor = grid.best_estimator_

In [44]:
predictor.feature_importances_

array([  30,    0, 3309,  862, 2529, 1667,    5,  125, 1109,  156,  192,
        128,   88,  207,  540,    0,  535,  850,   24,  534,  715,   82,
         18, 1206,  533,  985,   24,  743,  398,  170,  664,  746,  437,
          0,    0,  241,    0,    0,  278,  655,    0,  234,   72,  236,
        106,    0,    0,  205,   33,   42,   49,   17,   10,   58,    3,
        119,    0,   66,    1,   14,    0,  138,    0,    9,  124,    0,
         51,    0,   75,    3,   67,    1,   75,    3,    0,    0,    0,
          9,  129,    0,  722,  886,  555,  672,  885,  609,  527, 1148,
          3,  274,    0,    1,    1,    0,    0,    0,    0,    7,   23,
          0,    0,    0,    0,    0,    0,    2,   10,    0,    3,    5,
          4,    1,    0,    0, 1741, 1938])

In [45]:
predictor.fit(training_features,training_labels)

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.037644697126700453,
        max_bin=249, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_data_in_leaf=122, min_split_gain=0.0,
        n_estimators=359, n_jobs=-1, num_leaves=1447, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [46]:
prediction_submit = predictor.predict_proba(predict_features)[:,1]

In [47]:
predict_features['label']=prediction_submit

In [48]:
submit = predict.set_index('person')

In [49]:
submit['label'] = predict_features['label']

In [50]:
submit.to_csv('submit.csv')